In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import \
    train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import \
    MinMaxScaler, StandardScaler, \
    FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import \
    LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import \
    KNeighborsClassifier
from sklearn.tree import \
    DecisionTreeClassifier
from sklearn.ensemble import \
    RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans

from sklearn.neighbors import \
    KNeighborsRegressor
from sklearn.ensemble import \
    RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from pyspark.sql import SparkSession, \
    functions, types, Row

In [8]:
xa = np.random.normal(2, 1, 100)
xb = np.random.normal(3, 1, 100)
xc = xa + xb
df = pd.DataFrame({
    'xa': xa,
    'xb': xb,
    'xc': xc,
})

xa


array([ 2.79088513e+00,  1.33324027e+00,  1.53157605e+00,  1.62238859e+00,
        1.74878351e+00,  1.39529888e+00,  2.21309052e+00,  1.44522148e+00,
       -1.95002780e-02,  2.49748218e+00,  1.94728798e+00,  2.06081499e+00,
        2.77196376e+00,  2.32804054e+00,  7.42582364e-01,  2.82489997e+00,
        2.09240474e+00,  3.20867801e+00,  1.96873927e+00,  2.88065150e+00,
        1.94515306e+00,  2.36802298e+00,  2.36502353e+00,  2.53829674e+00,
        1.76546704e+00,  2.53625811e+00,  2.42199988e+00,  3.65440398e+00,
        8.04944243e-01,  1.65007531e+00,  2.84730517e+00,  1.15256208e+00,
        2.03777809e+00,  1.47650839e+00, -5.74684349e-01,  8.78724870e-01,
        3.25045433e+00,  1.04532862e+00,  2.25329086e+00,  2.99789535e+00,
        2.67114273e+00,  4.48026178e-01,  7.67766970e-01,  5.08130305e+00,
        2.11249484e+00,  4.40585713e+00,  1.75738319e+00,  6.72805369e-01,
        3.82310889e-01,  1.57314433e+00,  8.84134291e-01,  1.63663227e+00,
        7.76815066e-01,  

In [10]:
X = np.stack([xa, xb, xc], axis=1)
y = (xa + xb + xc) // 3
X_train, X_valid, y_train, y_valid \
    = train_test_split(X, y)
y

array([3., 2., 2., 2., 2., 3., 3., 3., 1., 4., 4., 4., 3., 3., 2., 4., 2.,
       4., 2., 3., 3., 2., 4., 3., 2., 5., 2., 5., 3., 2., 4., 1., 3., 3.,
       2., 3., 4., 2., 3., 3., 3., 1., 1., 5., 3., 3., 2., 2., 2., 2., 2.,
       2., 2., 2., 4., 2., 2., 5., 4., 2., 3., 4., 3., 2., 4., 2., 4., 3.,
       2., 3., 3., 3., 2., 3., 3., 3., 3., 4., 2., 2., 3., 3., 1., 2., 1.,
       0., 2., 3., 4., 2., 3., 2., 4., 3., 3., 4., 4., 2., 3., 2.])

In [ ]:
model = LinearRegression(
    fit_intercept=True)
model = GaussianNB()
model = make_pipeline(
    StandardScaler(),
    PCA(2),
    KNeighborsClassifier(5)
)
model = DecisionTreeClassifier(max_depth=4)
model = RandomForestClassifier(n_estimators=20,
        max_depth=3, min_samples_leaf=10)
model = MLPClassifier(hidden_layer_sizes=(4, 3))

model.fit(X_train, y_train)
print(model.predict(X_valid))
print(model.score(X_valid, y_valid))

model = KMeans(n_clusters=5)
y = model.fit_predict(X)
print(model.predict(X))


model = KNeighborsRegressor(5)
model = RandomForestRegressor(100, max_depth=5)
model = MLPRegressor(hidden_layer_sizes=(8, 6),
    activation='logistic')
                     
model.fit(X_train, y_train)
print(model.score(X_valid, y_valid))



In [11]:
spark = SparkSession.builder \
    .getOrCreate()

d0 = spark.createDataFrame(df).repartition(100)
d1 = d0.select(
    d0['xa'], d0['xb'],
    functions.round(d0['xa'],0).alias('r'),
)
grouped = d1.groupby('r').agg(
    functions.avg('xa'), functions.max('xb')
)
grouped = grouped.filter(grouped['r'] > 0)
grouped.show()

+---+------------------+-----------------+
|  r|           avg(xa)|          max(xb)|
+---+------------------+-----------------+
|1.0| 1.076238119029719| 4.95924855987185|
|4.0|3.8547895439720525|4.468617718841048|
|5.0| 5.081303045336318| 3.32705771379321|
|2.0| 2.011306865482866|5.009508849978685|
|3.0|2.8593896032407256|5.013441339104866|
+---+------------------+-----------------+



In [18]:
d2 = d1
d2=d2.filter(d2['xa']<0)
d2.show()


+--------------------+------------------+----+
|                  xa|                xb|   r|
+--------------------+------------------+----+
| -0.5746843491548566| 4.004009292336165|-1.0|
|-0.01950027798651366| 1.970549633783163| 0.0|
| -0.6209266952888828|1.8897834297636644|-1.0|
+--------------------+------------------+----+

